In [1]:
import gzip
import pandas as pd
import xml.etree.ElementTree as ET

# JMDICT

In [12]:
with gzip.open('./dicts/JMdict_e.gz') as f:
    lines = []
    for line in f:
        if not line == b'<JMdict>\n':
            lines.append(line)
        else:
            break

with open('./dicts/jmdict_doc.txt', mode='wb') as f:
    f.writelines(lines)

In [4]:
with gzip.open('./dicts/JMdict_e.gz', mode='r') as f:
    tree = ET.parse(f)

In [5]:
root = tree.getroot()
root[1000][0].tag

'ent_seq'

In [6]:
jmdict = []
for entry in root:
    new_entry = {}
    for element in entry:
        if element.tag == 'ent_seq':
            new_entry['id'] = element.text
        else:
            for sub_element in element:
                tag = sub_element.tag
                text = sub_element.text
                if tag in new_entry:
                    try:
                        new_entry[tag] += ', '+text
                    except Exception as e:
                        pass
                else:
                    new_entry[tag] = text
    jmdict.append(new_entry)


In [7]:
jmdict[104644]

{'id': '2138460',
 'reb': 'しめる',
 'pos': 'Ichidan verb, auxiliary verb',
 'xref': '使む, 成らしめる',
 'misc': 'archaic',
 's_inf': 'old causative verbal ending',
 'gloss': 'to make, to let'}

# CC-CEDICT

In [7]:
with gzip.open('./dicts/cedict_1_0_ts_utf-8_mdbg.txt.gz', mode='rt', encoding='utf8') as f:
    for _ in range(500):
        print(f.readline())

# CC-CEDICT

# Community maintained free Chinese-English dictionary.

# 

# Published by MDBG

# 

# License:

# Creative Commons Attribution-ShareAlike 4.0 International License

# https://creativecommons.org/licenses/by-sa/4.0/

# 

# Referenced works:

# CEDICT - Copyright (C) 1997, 1998 Paul Andrew Denisowski

# 

# CC-CEDICT can be downloaded from:

# https://www.mdbg.net/chinese/dictionary?page=cc-cedict

# 

# Additions and corrections can be sent through:

# https://cc-cedict.org/editor/editor.php

# 

# For more information about CC-CEDICT see:

# https://cc-cedict.org/wiki/

# 

#! version=1

#! subversion=0

#! format=ts

#! charset=UTF-8

#! entries=121862

#! publisher=MDBG

#! license=https://creativecommons.org/licenses/by-sa/4.0/

#! date=2023-10-12T03:57:48Z

#! time=1697083068

% % [pa1] /percent (Tw)/

2019冠狀病毒病 2019冠状病毒病 [er4 ling2 yi1 jiu3 guan1 zhuang4 bing4 du2 bing4] /COVID-19, the coronavirus disease identified in 2019/

21三體綜合症 21三体综合症 [er4 shi2 yi1 san1 ti3 z

In [8]:
lines = []
with gzip.open('./dicts/cedict_1_0_ts_utf-8_mdbg.txt.gz', mode='rt', encoding='utf8') as f:
    for line in f:
        if line.startswith('#'):
            lines.append(line)
        else:
            break
with open('./dicts/cedict_doc.txt', mode='w') as f:
    f.writelines(lines)